In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import dateutil
import statsmodels
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
sns.set()
pd.options.display.max_columns = None

In [2]:
claims_costs_path = "D:\Installed\Git_kraken_repos\Virtusa\datasets\claims\\"
ehr_path = "D:\Installed\Git_kraken_repos\Virtusa\datasets\Ehr\\"

In [3]:
ehr = {}
ehr_table_names = os.listdir(ehr_path)
for x in ehr_table_names:
    ehr[x[:-4]] = pd.read_csv(ehr_path+x,encoding = 'latin',low_memory=False)
ehr.keys()

dict_keys(['admissions', 'diagnoses', 'd_icd_procedures', 'icustays', 'patients', 'prescriptions', 'procedures_icd'])

In [4]:
claims_costs = pd.read_csv(claims_costs_path+'claims_to_ehr.csv')
claims_costs.head()

,AGE,LENGTH_OF_STAY,GENDER,RACE,AGE_BINS,GAP,LENGTH_OF_STAY_BINS,TOTAL_COST,PAYABLE,RATIO
0,67,4,Female,WHITE,"(62.5, 70.2]",4000.0,"(3.5, 7.0]",5024.0,1024.0,0.203822
1,75,4,Male,WHITE,"(70.2, 77.9]",4000.0,"(3.5, 7.0]",5024.0,1024.0,0.203822
2,77,2,Male,WHITE,"(70.2, 77.9]",11000.0,"(-0.035, 3.5]",12068.0,1068.0,0.088499
3,67,3,Female,WHITE,"(62.5, 70.2]",4000.0,"(-0.035, 3.5]",5068.0,1068.0,0.210734
4,70,0,Male,WHITE,"(62.5, 70.2]",6080.0,"(-0.035, 3.5]",7148.0,1068.0,0.149412


In [5]:
admi = ehr['admissions']
admi.dropna(inplace=True,subset=["diagnosis"])
admi

,admission_location,admission_type,admittime,deathtime,diagnosis,discharge_location,dischtime,edouttime,edregtime,ethnicity,hadm_id,has_chartevents_data,hospital_expire_flag,insurance,language,marital_status,religion,row_id,subject_id
0,EMERGENCY ROOM ADMIT,EMERGENCY,2196-04-09T12:26:00.000Z,NaN,BENZODIAZEPINE OVERDOSE,DISC-TRAN CANCER/CHLDRN H,2196-04-10T15:54:00.000Z,2196-04-09T13:24:00.000Z,2196-04-09T10:06:00.000Z,WHITE,165315,1,0,Private,NaN,MARRIED,UNOBTAINABLE,21,22
1,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2153-09-03T07:15:00.000Z,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2153-09-08T19:10:00.000Z,NaN,NaN,WHITE,152223,1,0,Medicare,NaN,MARRIED,CATHOLIC,22,23
2,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2157-10-18T19:34:00.000Z,NaN,BRAIN MASS,HOME HEALTH CARE,2157-10-25T14:00:00.000Z,NaN,NaN,WHITE,124321,1,0,Medicare,ENGL,MARRIED,CATHOLIC,23,23
3,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2139-06-06T16:14:00.000Z,NaN,INTERIOR MYOCARDIAL INFARCTION,HOME,2139-06-09T12:48:00.000Z,NaN,NaN,WHITE,161859,1,0,Private,NaN,SINGLE,PROTESTANT QUAKER,24,24
4,EMERGENCY ROOM ADMIT,EMERGENCY,2160-11-02T02:06:00.000Z,NaN,ACUTE CORONARY SYNDROME,HOME,2160-11-05T14:55:00.000Z,2160-11-02T04:27:00.000Z,2160-11-02T01:01:00.000Z,WHITE,129635,1,0,Private,NaN,MARRIED,UNOBTAINABLE,25,25
5,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2126-05-06T15:16:00.000Z,NaN,V-TACH,HOME,2126-05-13T15:00:00.000Z,NaN,NaN,UNKNOWN/NOT SPECIFIED,197661,1,0,Medicare,NaN,SINGLE,CATHOLIC,26,26
6,PHYS REFERRAL/NORMAL DELI,NEWBORN,2191-11-30T22:16:00.000Z,NaN,NEWBORN,HOME,2191-12-03T14:45:00.000Z,NaN,NaN,WHITE,134931,1,0,Private,NaN,NaN,CATHOLIC,27,27
7,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2177-09-01T07:15:00.000Z,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2177-09-06T16:00:00.000Z,NaN,NaN,WHITE,162569,1,0,Medicare,NaN,MARRIED,CATHOLIC,28,28
8,TRANSFER FROM HOSP/EXTRAM,URGENT,2172-10-14T14:17:00.000Z,NaN,UNSTABLE ANGINA\CATH,HOME HEALTH CARE,2172-10-19T14:37:00.000Z,NaN,NaN,UNKNOWN/NOT SPECIFIED,104557,1,0,Medicare,NaN,MARRIED,CATHOLIC,29,30
9,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2108-08-22T23:27:00.000Z,2108-08-30T15:00:00.000Z,STATUS EPILEPTICUS,DEAD/EXPIRED,2108-08-30T15:00:00.000Z,NaN,NaN,WHITE,128652,1,1,Medicare,NaN,MARRIED,CATHOLIC,30,31


In [6]:
searchfor = ["ANGINA","MYOCARDIAL","CORONARY ARTERY","ISCHEMIC HEART","ISCHEMIC HEART"]
columns_to_drop = ["row_id","deathtime","edregtime","edouttime","hospital_expire_flag","has_chartevents_data"]
df_ischemic = pd.merge(admi[admi.diagnosis.str.contains('|'.join(searchfor))].drop(columns = columns_to_drop),ehr['patients'],on='subject_id',how='left').drop(columns=["dod","dod_hosp","dod_ssn"])
df_ischemic.dropna(inplace=True,subset=["dob"])
df_ischemic

,admission_location,admission_type,admittime,diagnosis,discharge_location,dischtime,ethnicity,hadm_id,insurance,language,marital_status,religion,subject_id,dob,expire_flag,gender,row_id
0,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2153-09-03T07:15:00.000Z,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2153-09-08T19:10:00.000Z,WHITE,152223,Medicare,NaN,MARRIED,CATHOLIC,23,2082-07-17T00:00:00.000Z,0,M,20
1,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2139-06-06T16:14:00.000Z,INTERIOR MYOCARDIAL INFARCTION,HOME,2139-06-09T12:48:00.000Z,WHITE,161859,Private,NaN,SINGLE,PROTESTANT QUAKER,24,2100-05-31T00:00:00.000Z,0,M,21
2,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2177-09-01T07:15:00.000Z,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2177-09-06T16:00:00.000Z,WHITE,162569,Medicare,NaN,MARRIED,CATHOLIC,28,2103-04-15T00:00:00.000Z,0,M,25
3,TRANSFER FROM HOSP/EXTRAM,URGENT,2172-10-14T14:17:00.000Z,UNSTABLE ANGINA\CATH,HOME HEALTH CARE,2172-10-19T14:37:00.000Z,UNKNOWN/NOT SPECIFIED,104557,Medicare,NaN,MARRIED,CATHOLIC,30,1872-10-14T00:00:00.000Z,0,M,26
4,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2122-02-10T11:15:00.000Z,AORTIC VALVE DISEASE\CORONARY ARTERY BYPASS GR...,HOME HEALTH CARE,2122-02-20T15:30:00.000Z,WHITE,166707,Medicare,NaN,DIVORCED,CATHOLIC,35,2052-09-01T00:00:00.000Z,0,F,31
5,CLINIC REFERRAL/PREMATURE,EMERGENCY,2131-04-30T07:15:00.000Z,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2131-05-08T14:00:00.000Z,WHITE,182104,Medicare,ENGL,MARRIED,NOT SPECIFIED,36,2061-08-17T00:00:00.000Z,0,M,32
6,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2166-08-10T00:28:00.000Z,ACUTE MYOCARDIAL INFARCTION-SEPSIS,LONG TERM CARE HOSPITAL,2166-09-04T11:30:00.000Z,WHITE,185910,Medicare,NaN,WIDOWED,CATHOLIC,38,2090-08-31T00:00:00.000Z,0,M,34
7,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2176-06-30T03:00:00.000Z,R/O MYOCARDIAL INFARCTION,HOME,2176-07-03T15:12:00.000Z,UNKNOWN/NOT SPECIFIED,144265,Self Pay,VIET,MARRIED,BUDDHIST,359,2127-08-18T00:00:00.000Z,0,M,338
8,EMERGENCY ROOM ADMIT,EMERGENCY,2112-07-10T02:31:00.000Z,HYPONATREMIA-R/O MYOCARDIAL INFARCTION-RHABDOM...,SNF,2112-07-16T13:45:00.000Z,WHITE,142749,Medicare,NaN,WIDOWED,UNOBTAINABLE,362,2026-09-23T00:00:00.000Z,1,F,341
9,CLINIC REFERRAL/PREMATURE,EMERGENCY,2147-12-15T17:10:00.000Z,CORONARY ARTERY DISEASE\CARDIAC CATH,HOME,2147-12-23T12:17:00.000Z,WHITE,139873,Private,NaN,MARRIED,JEWISH,365,2088-02-23T00:00:00.000Z,1,M,344


In [7]:
def days_to_years( number_of_days ):
    res = int(number_of_days / 365)
    if res>0.0:
        return res
    else:
        return np.NAN
df_ischemic.admittime = pd.to_datetime(df_ischemic.admittime)
df_ischemic.dischtime = pd.to_datetime(df_ischemic.dischtime)
df_ischemic.dob = pd.to_datetime(df_ischemic.dob)
#df_ischemic["AGE"] = (df_ischemic.CLM_ADMSN_DT - df_ischemic.BENE_BIRTH_DT).dt.days.apply(days_to_years)
df_std = 100#df_ischemic["AGE"] = (df_ischemic.CLM_ADMSN_DT - df_ischemic.BENE_BIRTH_DT).dt.days.apply(days_to_years)
df_ischemic["LENGTH_OF_STAY"] = (df_ischemic.dischtime - df_ischemic.admittime ).dt.days
df_ischemic["AGE"] = (df_ischemic.admittime - df_ischemic.dob ).dt.days.apply(days_to_years)
df_ischemic.drop(columns=["dob"],inplace=True)
df_ischemic.dropna(inplace=True,subset=["AGE"])
df_ischemic

,admission_location,admission_type,admittime,diagnosis,discharge_location,dischtime,ethnicity,hadm_id,insurance,language,marital_status,religion,subject_id,expire_flag,gender,row_id,LENGTH_OF_STAY,AGE
0,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2153-09-03 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2153-09-08 19:10:00,WHITE,152223,Medicare,NaN,MARRIED,CATHOLIC,23,0,M,20,5,71.0
1,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2139-06-06 16:14:00,INTERIOR MYOCARDIAL INFARCTION,HOME,2139-06-09 12:48:00,WHITE,161859,Private,NaN,SINGLE,PROTESTANT QUAKER,24,0,M,21,2,39.0
2,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2177-09-01 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2177-09-06 16:00:00,WHITE,162569,Medicare,NaN,MARRIED,CATHOLIC,28,0,M,25,5,74.0
4,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2122-02-10 11:15:00,AORTIC VALVE DISEASE\CORONARY ARTERY BYPASS GR...,HOME HEALTH CARE,2122-02-20 15:30:00,WHITE,166707,Medicare,NaN,DIVORCED,CATHOLIC,35,0,F,31,10,69.0
5,CLINIC REFERRAL/PREMATURE,EMERGENCY,2131-04-30 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2131-05-08 14:00:00,WHITE,182104,Medicare,ENGL,MARRIED,NOT SPECIFIED,36,0,M,32,8,69.0
6,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2166-08-10 00:28:00,ACUTE MYOCARDIAL INFARCTION-SEPSIS,LONG TERM CARE HOSPITAL,2166-09-04 11:30:00,WHITE,185910,Medicare,NaN,WIDOWED,CATHOLIC,38,0,M,34,25,75.0
7,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2176-06-30 03:00:00,R/O MYOCARDIAL INFARCTION,HOME,2176-07-03 15:12:00,UNKNOWN/NOT SPECIFIED,144265,Self Pay,VIET,MARRIED,BUDDHIST,359,0,M,338,3,48.0
8,EMERGENCY ROOM ADMIT,EMERGENCY,2112-07-10 02:31:00,HYPONATREMIA-R/O MYOCARDIAL INFARCTION-RHABDOM...,SNF,2112-07-16 13:45:00,WHITE,142749,Medicare,NaN,WIDOWED,UNOBTAINABLE,362,1,F,341,6,85.0
9,CLINIC REFERRAL/PREMATURE,EMERGENCY,2147-12-15 17:10:00,CORONARY ARTERY DISEASE\CARDIAC CATH,HOME,2147-12-23 12:17:00,WHITE,139873,Private,NaN,MARRIED,JEWISH,365,1,M,344,7,59.0
10,CLINIC REFERRAL/PREMATURE,EMERGENCY,2192-11-19 18:14:00,CORONARY ARTERY DISEASE\CATH,SNF,2192-11-27 14:35:00,UNKNOWN/NOT SPECIFIED,181750,Medicare,ENGL,MARRIED,CATHOLIC,44,1,M,39,7,80.0


In [8]:
def to_gender(val):
    if val.gender=="M":
        return "Male"
    else:
        return "Female"
df_ischemic.gender = df_ischemic.apply(to_gender,axis=1)

In [9]:
def to_race(val):
    if val=="WHITE":
        return "WHITE"
    elif "BLACK" in val:
        return "BLACK"
    elif "HISPANIC" in val:
        return "HISPANIC"
    else:
        return "OTHERS"
df_ischemic.ethnicity = df_ischemic.ethnicity.apply(to_race)

In [10]:
claims_costs.head()

,AGE,LENGTH_OF_STAY,GENDER,RACE,AGE_BINS,GAP,LENGTH_OF_STAY_BINS,TOTAL_COST,PAYABLE,RATIO
0,67,4,Female,WHITE,"(62.5, 70.2]",4000.0,"(3.5, 7.0]",5024.0,1024.0,0.203822
1,75,4,Male,WHITE,"(70.2, 77.9]",4000.0,"(3.5, 7.0]",5024.0,1024.0,0.203822
2,77,2,Male,WHITE,"(70.2, 77.9]",11000.0,"(-0.035, 3.5]",12068.0,1068.0,0.088499
3,67,3,Female,WHITE,"(62.5, 70.2]",4000.0,"(-0.035, 3.5]",5068.0,1068.0,0.210734
4,70,0,Male,WHITE,"(62.5, 70.2]",6080.0,"(-0.035, 3.5]",7148.0,1068.0,0.149412


In [11]:
_,age_bins = pd.cut(claims_costs.AGE,bins=10,retbins=True)
_,length_bins = pd.cut(claims_costs.LENGTH_OF_STAY,bins=10,retbins=True)
df_ischemic["AGE_BINS"] = pd.cut(df_ischemic.AGE,bins=age_bins)
df_ischemic["LENGTH_OF_STAY_BINS"] = pd.cut(df_ischemic.LENGTH_OF_STAY,bins=length_bins)
df_ischemic.dropna(inplace=True,subset=["AGE_BINS","LENGTH_OF_STAY_BINS"])
df_ischemic

,admission_location,admission_type,admittime,diagnosis,discharge_location,dischtime,ethnicity,hadm_id,insurance,language,marital_status,religion,subject_id,expire_flag,gender,row_id,LENGTH_OF_STAY,AGE,AGE_BINS,LENGTH_OF_STAY_BINS
0,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2153-09-03 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2153-09-08 19:10:00,WHITE,152223,Medicare,NaN,MARRIED,CATHOLIC,23,0,Male,20,5,71.0,"(70.2, 77.9]","(3.5, 7.0]"
1,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2139-06-06 16:14:00,INTERIOR MYOCARDIAL INFARCTION,HOME,2139-06-09 12:48:00,WHITE,161859,Private,NaN,SINGLE,PROTESTANT QUAKER,24,0,Male,21,2,39.0,"(31.7, 39.4]","(-0.035, 3.5]"
2,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2177-09-01 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2177-09-06 16:00:00,WHITE,162569,Medicare,NaN,MARRIED,CATHOLIC,28,0,Male,25,5,74.0,"(70.2, 77.9]","(3.5, 7.0]"
4,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2122-02-10 11:15:00,AORTIC VALVE DISEASE\CORONARY ARTERY BYPASS GR...,HOME HEALTH CARE,2122-02-20 15:30:00,WHITE,166707,Medicare,NaN,DIVORCED,CATHOLIC,35,0,Female,31,10,69.0,"(62.5, 70.2]","(7.0, 10.5]"
5,CLINIC REFERRAL/PREMATURE,EMERGENCY,2131-04-30 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2131-05-08 14:00:00,WHITE,182104,Medicare,ENGL,MARRIED,NOT SPECIFIED,36,0,Male,32,8,69.0,"(62.5, 70.2]","(7.0, 10.5]"
6,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2166-08-10 00:28:00,ACUTE MYOCARDIAL INFARCTION-SEPSIS,LONG TERM CARE HOSPITAL,2166-09-04 11:30:00,WHITE,185910,Medicare,NaN,WIDOWED,CATHOLIC,38,0,Male,34,25,75.0,"(70.2, 77.9]","(24.5, 28.0]"
7,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2176-06-30 03:00:00,R/O MYOCARDIAL INFARCTION,HOME,2176-07-03 15:12:00,OTHERS,144265,Self Pay,VIET,MARRIED,BUDDHIST,359,0,Male,338,3,48.0,"(47.1, 54.8]","(-0.035, 3.5]"
8,EMERGENCY ROOM ADMIT,EMERGENCY,2112-07-10 02:31:00,HYPONATREMIA-R/O MYOCARDIAL INFARCTION-RHABDOM...,SNF,2112-07-16 13:45:00,WHITE,142749,Medicare,NaN,WIDOWED,UNOBTAINABLE,362,1,Female,341,6,85.0,"(77.9, 85.6]","(3.5, 7.0]"
9,CLINIC REFERRAL/PREMATURE,EMERGENCY,2147-12-15 17:10:00,CORONARY ARTERY DISEASE\CARDIAC CATH,HOME,2147-12-23 12:17:00,WHITE,139873,Private,NaN,MARRIED,JEWISH,365,1,Male,344,7,59.0,"(54.8, 62.5]","(3.5, 7.0]"
10,CLINIC REFERRAL/PREMATURE,EMERGENCY,2192-11-19 18:14:00,CORONARY ARTERY DISEASE\CATH,SNF,2192-11-27 14:35:00,OTHERS,181750,Medicare,ENGL,MARRIED,CATHOLIC,44,1,Male,39,7,80.0,"(77.9, 85.6]","(3.5, 7.0]"


In [12]:
def append_cost(row):
    '''
    Appends the cost to the individual patients in the ehr records based on the 4 parameters(Age,length of stay,race and gender)
    The best possible match will be found and a distribution of the matching parameters will be pulled from the claims database.
    A random sample will be pulled from this distribution and assigned to the row
    '''
    random_state = 42
    df = claims_costs[(claims_costs.LENGTH_OF_STAY_BINS == str(row.LENGTH_OF_STAY_BINS)) & (claims_costs.AGE_BINS == str(row.AGE_BINS)) & (claims_costs.GENDER == row.gender) & (claims_costs.RACE == row.ethnicity)][["TOTAL_COST","PAYABLE","GAP"]]
    a = np.random.randint(df_std)
    if(df.TOTAL_COST.count()>=1):
        x = df.sample(1,random_state=random_state)
        return str(str(x.TOTAL_COST.iloc[0] + a)+" "+str(x.PAYABLE.iloc[0]))
    else:
        df = claims_costs[(claims_costs.LENGTH_OF_STAY_BINS == str(row.LENGTH_OF_STAY_BINS)) & (claims_costs.AGE_BINS == str(row.AGE_BINS)) & (claims_costs.GENDER == row.gender)][["TOTAL_COST","PAYABLE","GAP"]]
        if(df.TOTAL_COST.count()>=1):
            x = df.sample(1,random_state=random_state)
            return str(str(x.TOTAL_COST.iloc[0] + a)+" "+str(x.PAYABLE.iloc[0]))
        else:
            df = claims_costs[(claims_costs.LENGTH_OF_STAY_BINS == str(row.LENGTH_OF_STAY_BINS)) & (claims_costs.AGE_BINS == str(row.AGE_BINS))][["TOTAL_COST","PAYABLE","GAP"]]
            if(df.TOTAL_COST.count()>=1):
                x = df.sample(1,random_state=random_state)
                return str(str(x.TOTAL_COST.iloc[0] + a)+" "+str(x.PAYABLE.iloc[0]))
            else:
                return np.NAN
df_ischemic["COSTS"] = df_ischemic[["AGE_BINS","LENGTH_OF_STAY_BINS","gender","ethnicity"]].apply(append_cost,axis=1)

In [13]:
df_ischemic["TOTAL_COST"],df_ischemic["PAYABLE"] = df_ischemic["COSTS"].str.split(' ', 1).str
df_ischemic.drop(columns=["COSTS"],inplace=True)
df_ischemic.dropna(inplace=True,subset=["TOTAL_COST","PAYABLE"])
df_ischemic.TOTAL_COST = df_ischemic.TOTAL_COST.astype('float')
df_ischemic.PAYABLE = df_ischemic.PAYABLE.astype('float')
df_ischemic.loc[df_ischemic["insurance"] == "Self Pay","PAYABLE"] = df_ischemic.loc[df_ischemic["insurance"] == "Self Pay","TOTAL_COST"]
df_ischemic["GAP"] = df_ischemic["TOTAL_COST"]-df_ischemic["PAYABLE"]
df_ischemic

,admission_location,admission_type,admittime,diagnosis,discharge_location,dischtime,ethnicity,hadm_id,insurance,language,marital_status,religion,subject_id,expire_flag,gender,row_id,LENGTH_OF_STAY,AGE,AGE_BINS,LENGTH_OF_STAY_BINS,TOTAL_COST,PAYABLE,GAP
0,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2153-09-03 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2153-09-08 19:10:00,WHITE,152223,Medicare,NaN,MARRIED,CATHOLIC,23,0,Male,20,5,71.0,"(70.2, 77.9]","(3.5, 7.0]",4437.0,1024.0,3413.0
1,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2139-06-06 16:14:00,INTERIOR MYOCARDIAL INFARCTION,HOME,2139-06-09 12:48:00,WHITE,161859,Private,NaN,SINGLE,PROTESTANT QUAKER,24,0,Male,21,2,39.0,"(31.7, 39.4]","(-0.035, 3.5]",6080.0,1024.0,5056.0
2,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2177-09-01 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2177-09-06 16:00:00,WHITE,162569,Medicare,NaN,MARRIED,CATHOLIC,28,0,Male,25,5,74.0,"(70.2, 77.9]","(3.5, 7.0]",4472.0,1024.0,3448.0
4,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2122-02-10 11:15:00,AORTIC VALVE DISEASE\CORONARY ARTERY BYPASS GR...,HOME HEALTH CARE,2122-02-20 15:30:00,WHITE,166707,Medicare,NaN,DIVORCED,CATHOLIC,35,0,Female,31,10,69.0,"(62.5, 70.2]","(7.0, 10.5]",8186.0,1100.0,7086.0
5,CLINIC REFERRAL/PREMATURE,EMERGENCY,2131-04-30 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2131-05-08 14:00:00,WHITE,182104,Medicare,ENGL,MARRIED,NOT SPECIFIED,36,0,Male,32,8,69.0,"(62.5, 70.2]","(7.0, 10.5]",7036.0,1024.0,6012.0
6,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2166-08-10 00:28:00,ACUTE MYOCARDIAL INFARCTION-SEPSIS,LONG TERM CARE HOSPITAL,2166-09-04 11:30:00,WHITE,185910,Medicare,NaN,WIDOWED,CATHOLIC,38,0,Male,34,25,75.0,"(70.2, 77.9]","(24.5, 28.0]",30131.0,1068.0,29063.0
7,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2176-06-30 03:00:00,R/O MYOCARDIAL INFARCTION,HOME,2176-07-03 15:12:00,OTHERS,144265,Self Pay,VIET,MARRIED,BUDDHIST,359,0,Male,338,3,48.0,"(47.1, 54.8]","(-0.035, 3.5]",3154.0,3154.0,0.0
8,EMERGENCY ROOM ADMIT,EMERGENCY,2112-07-10 02:31:00,HYPONATREMIA-R/O MYOCARDIAL INFARCTION-RHABDOM...,SNF,2112-07-16 13:45:00,WHITE,142749,Medicare,NaN,WIDOWED,UNOBTAINABLE,362,1,Female,341,6,85.0,"(77.9, 85.6]","(3.5, 7.0]",29056.0,1024.0,28032.0
9,CLINIC REFERRAL/PREMATURE,EMERGENCY,2147-12-15 17:10:00,CORONARY ARTERY DISEASE\CARDIAC CATH,HOME,2147-12-23 12:17:00,WHITE,139873,Private,NaN,MARRIED,JEWISH,365,1,Male,344,7,59.0,"(54.8, 62.5]","(3.5, 7.0]",7129.0,1068.0,6061.0
10,CLINIC REFERRAL/PREMATURE,EMERGENCY,2192-11-19 18:14:00,CORONARY ARTERY DISEASE\CATH,SNF,2192-11-27 14:35:00,OTHERS,181750,Medicare,ENGL,MARRIED,CATHOLIC,44,1,Male,39,7,80.0,"(77.9, 85.6]","(3.5, 7.0]",5465.0,1024.0,4441.0


Adding total length of stay in icu as a feature

In [14]:
sub_ids = df_ischemic.subject_id.unique()
icustays = ehr['icustays'][ehr['icustays'].subject_id.isin(sub_ids)].copy()
icustays.intime = pd.to_datetime(icustays.intime)
icustays.outtime = pd.to_datetime(icustays.outtime)
icustays.los.fillna(0,inplace=True)
icustays["ICU_LOS"] = icustays.los.apply(lambda x: round(x))
icustays

,dbsource,first_careunit,first_wardid,hadm_id,icustay_id,intime,last_careunit,last_wardid,los,outtime,row_id,subject_id,ICU_LOS
37,carevue,MICU,15,191028,249750,2133-03-29 02:34:11,MICU,15,1.8447,2133-03-30 22:50:35,402,302,2
41,carevue,CCU,7,133059,232719,2125-04-27 18:47:43,CCU,7,3.9185,2125-05-01 16:50:19,406,305,4
42,carevue,CSRU,15,108015,297009,2126-01-02 11:28:23,CSRU,15,4.9438,2126-01-07 10:07:29,407,305,5
43,carevue,MICU,50,122211,224571,2127-06-19 23:50:45,MICU,50,0.7925,2127-06-20 18:51:59,408,305,1
44,carevue,MICU,50,122211,232248,2127-07-03 20:02:34,MICU,50,5.8118,2127-07-09 15:31:36,409,305,6
45,metavision,SICU,33,194340,217232,2129-09-03 12:31:31,SICU,33,2.4370,2129-09-05 23:00:50,410,305,2
50,carevue,CCU,7,162308,211035,2160-06-04 09:45:55,CCU,7,1.3873,2160-06-05 19:03:42,415,309,1
55,carevue,CSRU,14,152144,266988,2177-04-29 08:04:53,CSRU,14,5.3387,2177-05-04 16:12:34,420,315,5
58,carevue,CCU,57,193264,225173,2129-06-07 08:50:00,CSRU,14,6.0799,2129-06-13 10:45:00,423,318,6
60,carevue,CCU,57,172849,221205,2145-10-09 09:50:48,CCU,57,0.4507,2145-10-09 20:39:48,425,320,0


In [15]:
icu_lenOfStay = pd.DataFrame(icustays.groupby(["subject_id","hadm_id"]).ICU_LOS.sum())
icu_lenOfStay.reset_index(inplace=True)
df_ischemic = pd.merge(df_ischemic,icu_lenOfStay,on=["subject_id","hadm_id"],how='left')
df_ischemic.ICU_LOS.fillna(0,inplace=True)
df_ischemic

,admission_location,admission_type,admittime,diagnosis,discharge_location,dischtime,ethnicity,hadm_id,insurance,language,marital_status,religion,subject_id,expire_flag,gender,row_id,LENGTH_OF_STAY,AGE,AGE_BINS,LENGTH_OF_STAY_BINS,TOTAL_COST,PAYABLE,GAP,ICU_LOS
0,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2153-09-03 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2153-09-08 19:10:00,WHITE,152223,Medicare,NaN,MARRIED,CATHOLIC,23,0,Male,20,5,71.0,"(70.2, 77.9]","(3.5, 7.0]",4437.0,1024.0,3413.0,1.0
1,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2139-06-06 16:14:00,INTERIOR MYOCARDIAL INFARCTION,HOME,2139-06-09 12:48:00,WHITE,161859,Private,NaN,SINGLE,PROTESTANT QUAKER,24,0,Male,21,2,39.0,"(31.7, 39.4]","(-0.035, 3.5]",6080.0,1024.0,5056.0,1.0
2,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2177-09-01 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2177-09-06 16:00:00,WHITE,162569,Medicare,NaN,MARRIED,CATHOLIC,28,0,Male,25,5,74.0,"(70.2, 77.9]","(3.5, 7.0]",4472.0,1024.0,3448.0,1.0
3,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2122-02-10 11:15:00,AORTIC VALVE DISEASE\CORONARY ARTERY BYPASS GR...,HOME HEALTH CARE,2122-02-20 15:30:00,WHITE,166707,Medicare,NaN,DIVORCED,CATHOLIC,35,0,Female,31,10,69.0,"(62.5, 70.2]","(7.0, 10.5]",8186.0,1100.0,7086.0,5.0
4,CLINIC REFERRAL/PREMATURE,EMERGENCY,2131-04-30 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2131-05-08 14:00:00,WHITE,182104,Medicare,ENGL,MARRIED,NOT SPECIFIED,36,0,Male,32,8,69.0,"(62.5, 70.2]","(7.0, 10.5]",7036.0,1024.0,6012.0,1.0
5,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2166-08-10 00:28:00,ACUTE MYOCARDIAL INFARCTION-SEPSIS,LONG TERM CARE HOSPITAL,2166-09-04 11:30:00,WHITE,185910,Medicare,NaN,WIDOWED,CATHOLIC,38,0,Male,34,25,75.0,"(70.2, 77.9]","(24.5, 28.0]",30131.0,1068.0,29063.0,26.0
6,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2176-06-30 03:00:00,R/O MYOCARDIAL INFARCTION,HOME,2176-07-03 15:12:00,OTHERS,144265,Self Pay,VIET,MARRIED,BUDDHIST,359,0,Male,338,3,48.0,"(47.1, 54.8]","(-0.035, 3.5]",3154.0,3154.0,0.0,3.0
7,EMERGENCY ROOM ADMIT,EMERGENCY,2112-07-10 02:31:00,HYPONATREMIA-R/O MYOCARDIAL INFARCTION-RHABDOM...,SNF,2112-07-16 13:45:00,WHITE,142749,Medicare,NaN,WIDOWED,UNOBTAINABLE,362,1,Female,341,6,85.0,"(77.9, 85.6]","(3.5, 7.0]",29056.0,1024.0,28032.0,1.0
8,CLINIC REFERRAL/PREMATURE,EMERGENCY,2147-12-15 17:10:00,CORONARY ARTERY DISEASE\CARDIAC CATH,HOME,2147-12-23 12:17:00,WHITE,139873,Private,NaN,MARRIED,JEWISH,365,1,Male,344,7,59.0,"(54.8, 62.5]","(3.5, 7.0]",7129.0,1068.0,6061.0,1.0
9,CLINIC REFERRAL/PREMATURE,EMERGENCY,2192-11-19 18:14:00,CORONARY ARTERY DISEASE\CATH,SNF,2192-11-27 14:35:00,OTHERS,181750,Medicare,ENGL,MARRIED,CATHOLIC,44,1,Male,39,7,80.0,"(77.9, 85.6]","(3.5, 7.0]",5465.0,1024.0,4441.0,4.0


Using procedures performed on the patients as a feature:-

In [16]:
d_icd_procedures = ehr["d_icd_procedures"].drop("row_id",axis=1)
d_icd_procedures.drop_duplicates(subset=["icd9_code"],keep='first',inplace=True)
d_icd_procedures

,icd9_code,long_title,short_title
0,851,Canthotomy,Canthotomy
1,852,Blepharorrhaphy,Blepharorrhaphy
2,859,Other adjustment of lid position,Adjust lid position NEC
3,861,Reconstruction of eyelid with skin flap or graft,Lid reconst w skin graft
4,862,Reconstruction of eyelid with mucous membrane ...,Lid reconst w muc graft
5,863,Reconstruction of eyelid with hair follicle graft,Lid reconst w hair graft
6,864,Reconstruction of eyelid with tarsoconjunctiva...,Lid recon-tarsoconj flap
7,869,Other reconstruction of eyelid with flaps or g...,Lid reconstr w graft NEC
8,870,"Reconstruction of eyelid, not otherwise specified",Lid reconstruction NOS
9,871,"Reconstruction of eyelid involving lid margin,...",Lid marg recon-part thic


In [17]:
procedures_icd = ehr["procedures_icd"].drop(["row_id","seq_num"],axis=1)
procedures_icd = procedures_icd.drop(procedures_icd.columns[0],axis=1)
procedures_icd

,hadm_id,icd9_code,subject_id
0,154460,3404,62641
1,130856,9671,2592
2,130856,3893,2592
3,119355,9672,55357
4,119355,331,55357
5,119355,3893,55357
6,158060,34,9545
7,189217,3613,28600
8,189217,3615,28600
9,189217,3961,28600


In [18]:
proc = pd.merge(procedures_icd,d_icd_procedures,on=["icd9_code"],how='left')
proc = proc[proc.subject_id.isin(sub_ids)]
proc

,hadm_id,icd9_code,subject_id,long_title,short_title
7,189217,3613,28600,(Aorto)coronary bypass of three coronary arteries,Aortocor bypas-3 cor art
8,189217,3615,28600,Single internal mammary-coronary artery bypass,1 int mam-cor art bypass
9,189217,3961,28600,Extracorporeal circulation auxiliary to open h...,Extracorporeal circulat
42,160689,3949,7671,Other revision of vascular procedure,Vasc proc revision NEC
43,160689,3722,7671,Left heart cardiac catheterization,Left heart cardiac cath
44,160689,8853,7671,Angiocardiography of left heart structures,Lt heart angiocardiogram
45,160689,8855,7671,Coronary arteriography using a single catheter,Coronar arteriogr-1 cath
46,141587,3612,16913,(Aorto)coronary bypass of two coronary arteries,Aortocor bypas-2 cor art
47,141587,3722,16913,Left heart cardiac catheterization,Left heart cardiac cath
48,141587,3961,16913,Extracorporeal circulation auxiliary to open h...,Extracorporeal circulat


In [19]:
proc.icd9_code.value_counts().head(10)

3961    3625
3615    3150
8856    2355
3722    1533
3612    1319
3723    1105
3613    1085
9904    1080
3893    1029
8853     949
Name: icd9_code, dtype: int64

Considering the top 10 procedures performed:-

In [20]:
top_procs = list(proc.icd9_code.value_counts().head(10).index)
proc = proc[proc.icd9_code.isin(top_procs)]
proc = pd.concat((proc,pd.get_dummies(proc.short_title)),axis=1)
proc

,hadm_id,icd9_code,subject_id,long_title,short_title,1 int mam-cor art bypass,Aortocor bypas-2 cor art,Aortocor bypas-3 cor art,Coronar arteriogr-2 cath,Extracorporeal circulat,Left heart cardiac cath,Lt heart angiocardiogram,Packed cell transfusion,Rt/left heart card cath,Venous cath NEC
7,189217,3613,28600,(Aorto)coronary bypass of three coronary arteries,Aortocor bypas-3 cor art,0,0,1,0,0,0,0,0,0,0
8,189217,3615,28600,Single internal mammary-coronary artery bypass,1 int mam-cor art bypass,1,0,0,0,0,0,0,0,0,0
9,189217,3961,28600,Extracorporeal circulation auxiliary to open h...,Extracorporeal circulat,0,0,0,0,1,0,0,0,0,0
43,160689,3722,7671,Left heart cardiac catheterization,Left heart cardiac cath,0,0,0,0,0,1,0,0,0,0
44,160689,8853,7671,Angiocardiography of left heart structures,Lt heart angiocardiogram,0,0,0,0,0,0,1,0,0,0
46,141587,3612,16913,(Aorto)coronary bypass of two coronary arteries,Aortocor bypas-2 cor art,0,1,0,0,0,0,0,0,0,0
47,141587,3722,16913,Left heart cardiac catheterization,Left heart cardiac cath,0,0,0,0,0,1,0,0,0,0
48,141587,3961,16913,Extracorporeal circulation auxiliary to open h...,Extracorporeal circulat,0,0,0,0,1,0,0,0,0,0
52,141587,3615,16913,Single internal mammary-coronary artery bypass,1 int mam-cor art bypass,1,0,0,0,0,0,0,0,0,0
54,141587,8853,16913,Angiocardiography of left heart structures,Lt heart angiocardiogram,0,0,0,0,0,0,1,0,0,0


In [21]:
proc_final = proc.groupby(['subject_id','hadm_id']).sum().reset_index().drop("icd9_code",axis=1)
proc_final.fillna(0,inplace=True)
proc_final

,subject_id,hadm_id,1 int mam-cor art bypass,Aortocor bypas-2 cor art,Aortocor bypas-3 cor art,Coronar arteriogr-2 cath,Extracorporeal circulat,Left heart cardiac cath,Lt heart angiocardiogram,Packed cell transfusion,Rt/left heart card cath,Venous cath NEC
0,13,143045,1,1,0,0,1,0,0,0,0,0
1,20,157681,1,1,0,0,1,0,0,0,0,0
2,23,152223,1,0,1,0,1,0,0,1,0,0
3,24,161859,0,0,0,1,0,0,0,0,1,0
4,28,162569,1,0,0,0,1,0,0,0,0,1
5,35,166707,1,0,0,0,1,0,0,1,0,0
6,36,122659,0,0,0,0,0,0,0,1,0,1
7,36,182104,1,0,0,1,1,1,1,0,0,0
8,38,185910,0,0,0,0,0,0,0,0,0,1
9,44,181750,1,1,0,1,1,1,0,1,0,0


In [22]:
df_ischemic = pd.merge(df_ischemic,proc_final,on=['subject_id','hadm_id'],how='left')

In [23]:
df_ischemic.iloc[:,23:] = df_ischemic.iloc[:,23:].fillna(0)
df_ischemic

,admission_location,admission_type,admittime,diagnosis,discharge_location,dischtime,ethnicity,hadm_id,insurance,language,marital_status,religion,subject_id,expire_flag,gender,row_id,LENGTH_OF_STAY,AGE,AGE_BINS,LENGTH_OF_STAY_BINS,TOTAL_COST,PAYABLE,GAP,ICU_LOS,1 int mam-cor art bypass,Aortocor bypas-2 cor art,Aortocor bypas-3 cor art,Coronar arteriogr-2 cath,Extracorporeal circulat,Left heart cardiac cath,Lt heart angiocardiogram,Packed cell transfusion,Rt/left heart card cath,Venous cath NEC
0,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2153-09-03 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2153-09-08 19:10:00,WHITE,152223,Medicare,NaN,MARRIED,CATHOLIC,23,0,Male,20,5,71.0,"(70.2, 77.9]","(3.5, 7.0]",4437.0,1024.0,3413.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2139-06-06 16:14:00,INTERIOR MYOCARDIAL INFARCTION,HOME,2139-06-09 12:48:00,WHITE,161859,Private,NaN,SINGLE,PROTESTANT QUAKER,24,0,Male,21,2,39.0,"(31.7, 39.4]","(-0.035, 3.5]",6080.0,1024.0,5056.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2177-09-01 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2177-09-06 16:00:00,WHITE,162569,Medicare,NaN,MARRIED,CATHOLIC,28,0,Male,25,5,74.0,"(70.2, 77.9]","(3.5, 7.0]",4472.0,1024.0,3448.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2122-02-10 11:15:00,AORTIC VALVE DISEASE\CORONARY ARTERY BYPASS GR...,HOME HEALTH CARE,2122-02-20 15:30:00,WHITE,166707,Medicare,NaN,DIVORCED,CATHOLIC,35,0,Female,31,10,69.0,"(62.5, 70.2]","(7.0, 10.5]",8186.0,1100.0,7086.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,CLINIC REFERRAL/PREMATURE,EMERGENCY,2131-04-30 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2131-05-08 14:00:00,WHITE,182104,Medicare,ENGL,MARRIED,NOT SPECIFIED,36,0,Male,32,8,69.0,"(62.5, 70.2]","(7.0, 10.5]",7036.0,1024.0,6012.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
5,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2166-08-10 00:28:00,ACUTE MYOCARDIAL INFARCTION-SEPSIS,LONG TERM CARE HOSPITAL,2166-09-04 11:30:00,WHITE,185910,Medicare,NaN,WIDOWED,CATHOLIC,38,0,Male,34,25,75.0,"(70.2, 77.9]","(24.5, 28.0]",30131.0,1068.0,29063.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2176-06-30 03:00:00,R/O MYOCARDIAL INFARCTION,HOME,2176-07-03 15:12:00,OTHERS,144265,Self Pay,VIET,MARRIED,BUDDHIST,359,0,Male,338,3,48.0,"(47.1, 54.8]","(-0.035, 3.5]",3154.0,3154.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,EMERGENCY ROOM ADMIT,EMERGENCY,2112-07-10 02:31:00,HYPONATREMIA-R/O MYOCARDIAL INFARCTION-RHABDOM...,SNF,2112-07-16 13:45:00,WHITE,142749,Medicare,NaN,WIDOWED,UNOBTAINABLE,362,1,Female,341,6,85.0,"(77.9, 85.6]","(3.5, 7.0]",29056.0,1024.0,28032.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,CLINIC REFERRAL/PREMATURE,EMERGENCY,2147-12-15 17:10:00,CORONARY ARTERY DISEASE\CARDIAC CATH,HOME,2147-12-23 12:17:00,WHITE,139873,Private,NaN,MARRIED,JEWISH,365,1,Male,344,7,59.0,"(54.8, 62.5]","(3.5, 7.0]",7129.0,1068.0,6061.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
9,CLINIC REFERRAL/PREMATURE,EMERGENCY,2192-11-19 18:14:00,CORONARY ARTERY DISEASE\CATH,SNF,2192-11-27 14:35:00,OTHERS,181750,Medicare,ENGL,MARRIED,CATHOLIC,44,1,Male,39,7,80.0,"(77.9, 85.6]","(3.5, 7.0]",5465.0,1024.0,4441.0,4.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0


Incorporating prescriptions as a feature:-

In [24]:
prescriptions = ehr['prescriptions'][ehr['prescriptions'].subject_id.isin(sub_ids)].copy()
prescriptions

,dose_unit_rx,dose_val_rx,drug,drug_name_generic,drug_name_poe,drug_type,enddate,form_unit_disp,form_val_disp,formulary_drug_cd,gsn,hadm_id,icustay_id,ndc,prod_strength,route,row_id,startdate,subject_id


In [25]:
prescriptions.hadm_id.value_counts()

Series([], Name: hadm_id, dtype: int64)

Very few subjects have take drugs and those who have taken,vary a lot. Adding each drug as a feature will just more noise

End of feature creation

# Final dataset:-

In [26]:
df_ischemic

,admission_location,admission_type,admittime,diagnosis,discharge_location,dischtime,ethnicity,hadm_id,insurance,language,marital_status,religion,subject_id,expire_flag,gender,row_id,LENGTH_OF_STAY,AGE,AGE_BINS,LENGTH_OF_STAY_BINS,TOTAL_COST,PAYABLE,GAP,ICU_LOS,1 int mam-cor art bypass,Aortocor bypas-2 cor art,Aortocor bypas-3 cor art,Coronar arteriogr-2 cath,Extracorporeal circulat,Left heart cardiac cath,Lt heart angiocardiogram,Packed cell transfusion,Rt/left heart card cath,Venous cath NEC
0,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2153-09-03 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2153-09-08 19:10:00,WHITE,152223,Medicare,NaN,MARRIED,CATHOLIC,23,0,Male,20,5,71.0,"(70.2, 77.9]","(3.5, 7.0]",4437.0,1024.0,3413.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2139-06-06 16:14:00,INTERIOR MYOCARDIAL INFARCTION,HOME,2139-06-09 12:48:00,WHITE,161859,Private,NaN,SINGLE,PROTESTANT QUAKER,24,0,Male,21,2,39.0,"(31.7, 39.4]","(-0.035, 3.5]",6080.0,1024.0,5056.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2177-09-01 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2177-09-06 16:00:00,WHITE,162569,Medicare,NaN,MARRIED,CATHOLIC,28,0,Male,25,5,74.0,"(70.2, 77.9]","(3.5, 7.0]",4472.0,1024.0,3448.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2122-02-10 11:15:00,AORTIC VALVE DISEASE\CORONARY ARTERY BYPASS GR...,HOME HEALTH CARE,2122-02-20 15:30:00,WHITE,166707,Medicare,NaN,DIVORCED,CATHOLIC,35,0,Female,31,10,69.0,"(62.5, 70.2]","(7.0, 10.5]",8186.0,1100.0,7086.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,CLINIC REFERRAL/PREMATURE,EMERGENCY,2131-04-30 07:15:00,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,HOME HEALTH CARE,2131-05-08 14:00:00,WHITE,182104,Medicare,ENGL,MARRIED,NOT SPECIFIED,36,0,Male,32,8,69.0,"(62.5, 70.2]","(7.0, 10.5]",7036.0,1024.0,6012.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
5,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2166-08-10 00:28:00,ACUTE MYOCARDIAL INFARCTION-SEPSIS,LONG TERM CARE HOSPITAL,2166-09-04 11:30:00,WHITE,185910,Medicare,NaN,WIDOWED,CATHOLIC,38,0,Male,34,25,75.0,"(70.2, 77.9]","(24.5, 28.0]",30131.0,1068.0,29063.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,2176-06-30 03:00:00,R/O MYOCARDIAL INFARCTION,HOME,2176-07-03 15:12:00,OTHERS,144265,Self Pay,VIET,MARRIED,BUDDHIST,359,0,Male,338,3,48.0,"(47.1, 54.8]","(-0.035, 3.5]",3154.0,3154.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,EMERGENCY ROOM ADMIT,EMERGENCY,2112-07-10 02:31:00,HYPONATREMIA-R/O MYOCARDIAL INFARCTION-RHABDOM...,SNF,2112-07-16 13:45:00,WHITE,142749,Medicare,NaN,WIDOWED,UNOBTAINABLE,362,1,Female,341,6,85.0,"(77.9, 85.6]","(3.5, 7.0]",29056.0,1024.0,28032.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,CLINIC REFERRAL/PREMATURE,EMERGENCY,2147-12-15 17:10:00,CORONARY ARTERY DISEASE\CARDIAC CATH,HOME,2147-12-23 12:17:00,WHITE,139873,Private,NaN,MARRIED,JEWISH,365,1,Male,344,7,59.0,"(54.8, 62.5]","(3.5, 7.0]",7129.0,1068.0,6061.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
9,CLINIC REFERRAL/PREMATURE,EMERGENCY,2192-11-19 18:14:00,CORONARY ARTERY DISEASE\CATH,SNF,2192-11-27 14:35:00,OTHERS,181750,Medicare,ENGL,MARRIED,CATHOLIC,44,1,Male,39,7,80.0,"(77.9, 85.6]","(3.5, 7.0]",5465.0,1024.0,4441.0,4.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0


Dropping several unnecessary columns:-

In [27]:
df_ischemic_final = df_ischemic.drop(columns=["subject_id","hadm_id","admittime","dischtime","diagnosis","row_id","expire_flag","language"])


Saving the dataset to use for building the predictive model

In [28]:
df_ischemic_final.to_csv("D:\Installed\Git_kraken_repos\Virtusa\datasets\Final\dataset.csv",index=False)